# Crew AI Project: AI outreach sales agents

### Libraries

In [ ]:
from crewai import Agent, Task, Crew, LLM
from dotenv import dotenv_values
from crewai.tools import BaseTool
from textblob import TextBlob
import os
import re

# Load environment variables from .env
config = dotenv_values(".env")

os.environ["SERPER_API_KEY"] = config.get("SERPER_API_KEY", "")
os.environ["MISTRAL_API_KEY"] = config.get("MISTRAL_API_KEY", "")

In [ ]:
"""
llm = LLM(
    model="groq/llama3-8b-8192",
    temperature=0.7,
    api_key=config['GROQ_API_KEY'],
)
"""

### Load LLM

In [ ]:
llm = LLM(
    model="mistral/mistral-large-latest",
    temperature=0.7,
    api_key=os.environ["MISTRAL_API_KEY"],
)

### Define agents using role and goals

In [ ]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

###

lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

###

### Extracting company info and sentiment analysis

In [ ]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

###

directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

###
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        blob = TextBlob(text)
        sentiment_polarity = blob.sentiment.polarity

        if sentiment_polarity > 0:
            return "positive"
        elif sentiment_polarity < 0:
            return "negative"
        else:
            return "neutral"
        
###

sentiment_analysis_tool = SentimentAnalysisTool()

lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

###

### Personalized Outreach

In [ ]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

### Implement the crew

In [ ]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=True
)

### Test Case

In [86]:
inputs = {
    "lead_name": "AIConsult.group GmbH",
    "industry": "AI Engineering & Project Management",
    "key_decision_maker": "Nicolas Bartschat",
    "position": "CEO",
    "milestone": "Project Completion or New Project Initiation"
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: abbbeef5-b86c-4677-9fdf-2cf3ec097aba                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b3bad206-dfc8-4003-81b5-657a2d96c15c
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b3bad206-dfc8-4003-81b5-657a2d96c15c
       Status: Executing Task...
    └── 🤖 Agent: Sales Representative
            Status: In Progress

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of AIConsult.group GmbH, a company in the AI Engineering & Project Management sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


🤖 Agent: Sales Representative
    Status: In Progress

...



# Agent: Lead Sales Representative
## Final Answer: 
---

**Email Draft 1: Introduction and Congratulations**

Subject: Congratulations on Your Recent Milestones & Exciting Opportunities Ahead

Dear Nicolas Bartschat,

I hope this message finds you well. I wanted to reach out and congratulate you and the AIConsult.group GmbH team on your recent achievements. Your dedication to integrating AI into daily business processes is truly inspiring, and it's clear that your efforts are paying off.

At CrewAI, we share your passion for leveraging advanced AI solutions to drive business success. We believe that our cutting-edge tools can complement your ongoing projects and help you achieve even greater efficiency and innovation.

I would love the opportunity to discuss how our solutions can support your goals and explore potential collaborations. Please let me know if you're available for a quick call or a demo in the coming weeks.

Best regards,

[Your Name]
[Your Position]
CrewAI

---

**Ema

🚀 Crew: crew
├── 📋 Task: b3bad206-dfc8-4003-81b5-657a2d96c15c
│      Assigned to: Sales Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Representative
│           Status: ✅ Completed
└── 📋 Task: 506b1543-ac5b-4bbb-8231-f81b1390903e
       Status: Executing Task...
    └── 🤖 Agent: Lead Sales Representative
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: b3bad206-dfc8-4003-81b5-657a2d96c15c
│      Assigned to: Sales Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Representative
│           Status: ✅ Completed
└── 📋 Task: 506b1543-ac5b-4bbb-8231-f81b1390903e
       Assigned to: Lead Sales Representative
       Status: ✅ Completed
    └── 🤖 Agent: Lead Sales Representative
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 506b1543-ac5b-4bbb-8231-f81b1390903e                                                                     │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: abbbeef5-b86c-4677-9fdf-2cf3ec097aba                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
Agents = [] 
Tasks = [] 
Responses = []

for task in result.tasks_output:
    Agents.append(task.agent)
    Tasks.append(task.description)
    Responses.append(str(task))

import re

def format_response(response_text):
    # 1. Convert markdown-style bold (**text**) to HTML <strong> tags
    formatted = re.sub(r'\*\*(.*?)\*\*', r'<strong>\1</strong>', response_text)
    
    # 2. Insert a <br> before numbered items (e.g. "1. ", "2. ")
    formatted = re.sub(r'(\d+\.\s)', r'<br>\1', formatted)
    
    # 3. Insert a <br> before email-specific sections:
    #    - "Email Draft <number>:"
    formatted = re.sub(r'(Email Draft\s*\d+:)', r'<br>\1', formatted)
    #    - "Subject:" keyword
    formatted = re.sub(r'(\bSubject:\b)', r'<br>\1', formatted)
    #    - "Dear" greeting
    formatted = re.sub(r'(\bDear\b)', r'<br>\1', formatted)
    #    - "Best regards," closing
    formatted = re.sub(r'(\bBest regards,)', r'<br>\1', formatted)
    
    
    # 4. Optionally, handle list items starting with a dash (if not already on a new line)
    # This finds a newline followed by any spaces and then a dash, and ensures a <br> is present.
    formatted = re.sub(r'(\n\s*-\s?)', r'<br>- ', formatted)
    
    # 5. Remove any extra leading <br> if the very first element matches
    formatted = re.sub(r'^<br>', '', formatted)
    
    return formatted

# Assume Responses is a list of strings (each can contain email drafts or paragraphs)
for i in range(len(Responses)):
    Responses[i] = format_response(Responses[i])

# Build the tasks section dynamically
tasks_section = ""
for i in range(len(Tasks)):
    tasks_section += f"""
    <div class="task-block">
        <div class="agent">Agent {i+1}: {Agents[i]}</div>
        <div class="task-desc"><strong>Task:</strong> {Tasks[i]}</div>
        <div class="response"><strong>Response:</strong> {Responses[i]}</div>
    </div>
    """

# HTML template for the one-page report
html_template = """
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Agent Task Report</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 40px; background-color: #f9f9f9; }}
        h1 {{ color: #333; }}
        .task-block {{ background: #fff; border: 1px solid #ddd; padding: 15px; margin-bottom: 20px; border-radius: 8px; }}
        .agent {{ font-weight: bold; color: #005a9c; }}
        .task-desc {{ margin-top: 5px; }}
        .response {{ margin-top: 10px; color: #444; }}
        .footer {{ margin-top: 40px; font-size: 0.9em; color: #777; }}
    </style>
</head>
<body>
    <h1>Agent Task Report</h1>
    {tasks_section}
    <div class="footer">
        <strong>Total Token Usage:</strong> {token_usage}
    </div>
</body>
</html>
"""

# Format the final HTML content with the tasks section and token usage
html_content = html_template.format(tasks_section=tasks_section, token_usage=result.token_usage)

# Save the HTML content to a file
with open("agent_task_report.html", "w", encoding="utf-8") as file:
    file.write(html_content)

print("Report saved as 'agent_task_report.html'")

Report saved as 'agent_task_report.html'
